# XAS Workflow Task 1 Batch processing

This notebook contains a modified version of the first task of the XAS processing workflow. In this case, the  

The break up of the task consist of the following steps 

| Task                            | Input                                         | Output
| -------------                   |-------------                                  | -----  
| Create Athena project      |                                               | 
| 1. Import data                |Files: 4000 Data \[.dat\] files                      | 
| 2. Normalisation              |No need as all data has been normalised in DAWN |
| 3. Save Athena Project        |                                               |Files: 4000 Athena \[.prj\] filwa

For more details about larch, see https://xraypy.github.io/xraylarch/xafs/

## Column and group names.

Knowing the elements of the input data set it is possible to start manipulating and visualising the XAFS data. 

In this case energy and $\mu$ are provided, but the names of the columns do not match the names used by Larch. We rename the columns. The name of the groups is the same as the name of the file. To make things easier we can also adjust the file name.


In [10]:
# Larch Libraries
# library to read ascii files
from larch.io import read_ascii
# libraries to handle data groups
from larch.utils import group2dict, dict2group
# library to normalise data
from larch.xafs import pre_edge
# import the larch.io libraries for managing athena files
from larch.io import create_athena, read_athena, extract_athenagroup

# File handling
from pathlib import Path

#plotting library
%matplotlib inline
import matplotlib.pyplot as plt

#library for writing to log
import logging

# initialise log file
def set_logger(log_file):
    logger = logging.getLogger()
    fhandler = logging.FileHandler(filename=log_file, mode='a')
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fhandler.setFormatter(formatter)
    logger.addHandler(fhandler)
    logger.setLevel(logging.DEBUG)

#reading all with the same extension files from a dir
def get_files_list(source_dir, f_pattern):
    i_counter = 0
    files_list = []
    for filepath in sorted(source_dir.glob(f_pattern)):
        i_counter += 1
        files_list.append(filepath)
    return files_list

# Rename columns 
def rename_cols(xafs_group):
    # energy
    engy = xafs_group.col1
    # mu
    mu_e = xafs_group.col2
    # get a dictionary from te group
    xafs_dict = group2dict(xafs_group)
    # add mu and energy to the dictionary
    xafs_dict['energy'] = engy
    xafs_dict['mu'] = mu_e
    xafs_group = dict2group(xafs_dict)
    return xafs_group

# show plot of normalised data
def plot_normalised(xafs_group):
        plt.plot(xafs_group.energy, xafs_group.pre_edge, 'g', label='pre-edge') # plot pre-edge in green
        plt.plot(xafs_group.energy, xafs_group.post_edge, 'r', label='post-edge')# plot post-edge in green
        plt.plot(xafs_group.energy, xafs_group.mu, 'b', label=xafs_group.filename) # plot mu in blue
        plt.grid(color='r', linestyle=':', linewidth=1) #show and format grid
        plt.xlabel('Energy (eV)') # label y graph
        plt.ylabel('x$\mu$(E)') # label y axis
        plt.title("pre-edge and post_edge fitting to $\mu$")
        plt.legend() # show legend

# variables that can be changed to process different datasets
data_path = "..\\nexusdata\\rh4co_ox_53\\37123_Rh_4_CO_Oxidation_45_7_ascii\\"
file_pattern = "*.dat"
f_prefix = "rh4co"
top_count = 10
i_count = 0
show_graph = False # False to prevent showing graphs


# create the path for storing results
base_path = Path("./" , f_prefix)
Path(base_path).mkdir(parents=True, exist_ok=True)

log_file = Path("./",base_path,"process.log")
print(log_file)
# set path for log
set_logger(log_file)

source_path = Path(data_path)
files_list = get_files_list(source_path, file_pattern)
xas_data = {}
logging.info("Started processing")
for a_file in files_list:
    file_name = a_file.name
    
    logging.info ("Processing: " + file_name)
    logging.info ("Path: "+ str(a_file))
    f_suffix = str(i_count).zfill(6) 
    p_name = f_prefix+f_suffix
    logging.info ("project name: "+ p_name)
    p_path = Path(base_path , p_name + ".prj")
    logging.info ("project path: "+ str(p_path))
    xas_data = read_ascii(a_file)
    # using vars(fe_xas) we see that the object has the following properties: 
    # path, filename, header, data, attrs, energy, xmu, i0
    # print(vars(xas_data))

    # rename columns and group
    xas_data = rename_cols(xas_data)
    # the group is the same as the file name
    xas_data.filename = p_name
    
    # calculate pre-edge and post edge and add them to group
    # using defaults
    pre_edge(energy=xas_data.energy, mu=xas_data.mu , group=xas_data)
    # Show graph if allowed
    if show_graph:
        plot_normalised(xas_data)
    
    xas_project = create_athena(p_path)
    xas_project.add_group(xas_data)
    xas_project.save()
    
    i_count +=1
    if i_count == top_count:
        break
       
logging.info("Finished processing")

[matplotlib.pyplot] DEBUG : Loaded backend module://ipykernel.pylab.backend_inline version unknown.


rh4co\process.log


[root] INFO : Started processing
[root] INFO : Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00000.dat
[root] INFO : Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00000.dat
[root] INFO : project name: rh4co000000
[root] INFO : project path: rh4co\rh4co000000.prj
[root] INFO : Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00001.dat
[root] INFO : Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00001.dat
[root] INFO : project name: rh4co000001
[root] INFO : project path: rh4co\rh4co000001.prj
[root] INFO : Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00002.dat
[root] INFO : Path: ..\nexusdata\rh4co_ox_53\37123_Rh_4_CO_Oxidation_45_7_ascii\37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00002.dat
[root] INFO : project name: rh4co000002
[root] INFO : project path: rh4co\rh4co000002.prj
[root] INFO : Processing: 37123_Rh_4_CO_Oxidation_45_7_rh4co_ox_47_00003.

In [8]:
logging.error('hello!')
logging.debug('This is a debug message')
logging.info('this is an info message')
logging.warning('tbllalfhldfhd, warning.')

[root] ERROR : hello!
[root] DEBUG : This is a debug message
[root] INFO : this is an info message
[root] WARNING : tbllalfhldfhd, warning.
